In [2]:
import json
import re

In [3]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

In [4]:
## Web search
search = DuckDuckGoSearchRun()

## Ollama
llm = ChatOllama(model="llama2")

## CSV Parser
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [5]:
# 템플릿 생성
template = """
Information: 
{information}
###
Question: 
{query}
###
Output format:
{format_instructions}
###
System:
{system}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions":  format_instructions}
    )

chain = prompt | llm | output_parser

In [6]:
national = "Korean"
facet = "food" # food, drink, clothing

In [16]:
system_prompt = "Limit speaking to anything beyond what is asked. !!DO NOT SPEAK SURE!!"

search_result = search.run(f"{national} traditional {facet}")
# 덕덕고 search 기능 파악 필요

query = f"Find proper nouns of traditional {national} {facet} from 'information' and write them according to the 'output format', excluding names that are general {facet}."

c:\Anaconda3\envs\PTLLM\lib\asyncio\events.py:81: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


In [17]:
print(search_result)

1: Chuncheon Dakgalbi - Spicy Stir Fried Chicken. Chuncheon dakgalbi is another of those traditional Korean dishes for those people who love fried meat. This dish combines marinated chicken with vegetables, rice cake (tteok 떡 ), chilli paste sauce, and other spices. You can also add noodles and cheese. 08 of 10. Seollangtang (Korean Ox Bone Soup) Debbi Smirnoff / Getty Images. Ox bone soup is an easy recipe, but it's one that you will simmer all day. Simmering the leg bones for several hours results in a milky-white, rich and meaty soup with garlic, ginger, and noodles. Get the Recipe. Korean cuisine is largely based on rice, vegetables, seafood and (at least in South Korea) meats. Dairy is largely absent from the traditional Korean diet. [3] Traditional Korean meals are named for the number of side dishes ( 반찬; 飯饌; banchan) that accompany steam-cooked short-grain rice. Bibimbap: This Korean lunch-in-a-bowl mixes together a simple salad of rice, mixed vegetables, rice, beef and egg, wi

In [19]:
results = set(chain.invoke({"system": system_prompt, "information": search_result, "query": query}))

print(search_result)
print()
print(results)
print()

# 추가할 항목을 담을 집합을 준비합니다.
proc_set = set()

for item in results:
    # 특수 문자 제거, 소문자 변환, 앞뒤 공백 제거
    cleaned_item = re.sub(r'[^a-zA-Z0-9\n\s]', '', item).lower().strip()

    # '\n'이 있는 경우 분할하여 저장
    if '\n' in cleaned_item:
        cleaned_item_list = cleaned_item.split('\n')
        for c in cleaned_item_list:
            proc_set.add(c.strip())
    else:
        proc_set.add(cleaned_item)

# 키워드 길이 0~20까지만 유지
filtered_results = {item for item in proc_set if 0 < len(item) < 20}

print(filtered_results)

1: Chuncheon Dakgalbi - Spicy Stir Fried Chicken. Chuncheon dakgalbi is another of those traditional Korean dishes for those people who love fried meat. This dish combines marinated chicken with vegetables, rice cake (tteok 떡 ), chilli paste sauce, and other spices. You can also add noodles and cheese. 08 of 10. Seollangtang (Korean Ox Bone Soup) Debbi Smirnoff / Getty Images. Ox bone soup is an easy recipe, but it's one that you will simmer all day. Simmering the leg bones for several hours results in a milky-white, rich and meaty soup with garlic, ginger, and noodles. Get the Recipe. Korean cuisine is largely based on rice, vegetables, seafood and (at least in South Korea) meats. Dairy is largely absent from the traditional Korean diet. [3] Traditional Korean meals are named for the number of side dishes ( 반찬; 飯饌; banchan) that accompany steam-cooked short-grain rice. Bibimbap: This Korean lunch-in-a-bowl mixes together a simple salad of rice, mixed vegetables, rice, beef and egg, wi

In [12]:
## 증강할 캡션 문장 로드
with open('data/sentences.json', 'r') as f:
    sentences = json.load(f)

print(sentences)

{'food': ['The young man is stirring his pot of {national_keyword} with a wooden spoon.', 'A group of people preparing {national_keyword} in a kitchen.', 'A kitchen counter with cutting board, knife and {national_keyword}.', 'A married couple preparing {national_keyword} in a house kitchen.', 'Several people are sitting around an outdoor table eating a {national_keyword}.', 'A man sitting at a table having a {national_keyword}.', 'A plate of {national_keyword} in containers is on a tray.', 'a close up of a few plates of {national_keyword} on a table', '{national_keyword} is served on a plate near a vase.', 'A man sitting at a table with a large plate of {national_keyword} on it.'], 'drink': ['Man in cycling clothes {national_keyword} beside his bicycle.', 'a plate of food on a table next to a {national_keyword}', 'Friends having a {national_keyword} together in the kitchen', 'Women are preparing {national_keyword} for themselves in the kitchen.', 'many people at tables with {national_k

In [13]:
for sentence in sentences[facet]:
    for result in filtered_results:
        augmented_sentence = sentence.format(national_keyword=result)
        print(augmented_sentence)


The young man is stirring his pot of la galbi with a wooden spoon.
The young man is stirring his pot of chuncheon dakgalbi with a wooden spoon.
The young man is stirring his pot of seollangtang with a wooden spoon.
A group of people preparing la galbi in a kitchen.
A group of people preparing chuncheon dakgalbi in a kitchen.
A group of people preparing seollangtang in a kitchen.
A kitchen counter with cutting board, knife and la galbi.
A kitchen counter with cutting board, knife and chuncheon dakgalbi.
A kitchen counter with cutting board, knife and seollangtang.
A married couple preparing la galbi in a house kitchen.
A married couple preparing chuncheon dakgalbi in a house kitchen.
A married couple preparing seollangtang in a house kitchen.
Several people are sitting around an outdoor table eating a la galbi.
Several people are sitting around an outdoor table eating a chuncheon dakgalbi.
Several people are sitting around an outdoor table eating a seollangtang.
A man sitting at a table